In [ ]:
import neo4j
import pandas as pd
from utils import simplify_to_centroid_if_small
from utils_neo4j import init, station_insert_query
from geofox_client import get_geofox_client
from utils_geofox import get_stations, stationdf2rows, get_departures, process_departure_data
import pandas as pd
import json

c:\Users\paul-\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Fetch the POIs
stations = get_stations(None, loadFromDisk=True)['stations']

In [4]:
stations_hamburg = [s for s in stations if s['city'] == 'Hamburg']

In [5]:
departures_all = []

# iterate over the stations in step size 5
for i in range(0, len(stations_hamburg), 5):
    sub_req = stations_hamburg[i:i+5]
    # set file name to join of the ids
    file_name = f"{i}_{i+5}.json"
    res = get_departures(None, sub_req, filename=file_name, loadFromDisk=True)
    departures_all += res['departures']


In [6]:
len(departures_all)

644327

In [7]:
departures_all_preproc = process_departure_data(departures_all)

Processing basic departure information...


Basic processing: 100%|██████████| 644327/644327 [00:02<00:00, 291824.38it/s]


Converting to pandas DataFrame...
Sorting departures chronologically...
Creating lookup structures...


Creating lookup: 100%|██████████| 644327/644327 [00:17<00:00, 37192.67it/s]


Finding next stations and calculating arrival times (in parallel)...


FIND_NEXT_STATION DONE: 100%|██████████| 644327/644327 [00:37<00:00, 17154.33it/s]


Cleaning up temporary data...
Processing complete. Processed 644327 departures.


In [10]:
departures_all_preproc

[{'from_station': 'Berliner Tor',
  'from_station_id': 'Master:10952',
  'line_terminal': 'Jungfernstieg',
  'line_origin': 'Hammer Kirche',
  'line_id': 'HHA-U:U2_HHA-U',
  'line_name': 'U2',
  'line_info': 'U-Bahn',
  'direction_id': 6,
  'departure_time': '2025-04-03 00:01',
  'platform': 'Gleis 3',
  'next_station': 'Hammer Kirche',
  'arrival_time': '2025-04-03 00:02'},
 {'from_station': 'Berliner Tor',
  'from_station_id': 'Master:10952',
  'line_terminal': 'Hammer Kirche',
  'line_origin': 'Jungfernstieg',
  'line_id': 'HHA-U:U2_HHA-U',
  'line_name': 'U2',
  'line_info': 'U-Bahn',
  'direction_id': 1,
  'departure_time': '2025-04-03 00:02',
  'platform': 'Gleis 2',
  'next_station': 'Burgstraße',
  'arrival_time': '2025-04-03 00:04'},
 {'from_station': 'Berliner Tor',
  'from_station_id': 'Master:10952',
  'line_terminal': 'Ohlsdorf',
  'line_origin': 'Elbgaustraße',
  'line_id': 'SBH:S5_SBH_SBAHNS',
  'line_name': 'S5',
  'line_info': 'S-Bahn',
  'direction_id': 1,
  'departur

In [ ]:
# safe departures preproc to json
with open('data_geofox/departures_all_preproc.json', 'w') as f:
    json.dump(departures_all_preproc, f)